In [ ]:
FILENAME = "siyasi.mp4" # video
SEARCH = "millet"       # aranan kelime

# Kurulumlar

In [ ]:
!pip install --quiet ipython-autotime
%load_ext autotime

!pip install --quiet ffmpeg-python==0.2.0

# Kütüphaneleri İçe Aktar

In [ ]:
from getpass import getpass
import requests
import ffmpeg
import re
import os
from pprint import pprint

# Core Functions

In [ ]:
def extract_sound_file(fileName):
    """
    Bir MP4 dosyasından sesi çıkarıp MP3 formatında kaydeder.

    Args:
        fileName (str): Sesin çıkarılacak MP4 dosyasının adı.

    Returns:
        str: Oluşturulan MP3 dosyasının adı.
    """

    # MP4 dosya adını MP3 formatına dönüştür
    audiofilename = fileName.replace(".mp4", '.mp3')

    # ffmpeg ile dosyayı okumak için giriş akışı oluştur
    input_stream = ffmpeg.input(fileName)

    # Giriş akışından sadece ses verisini al
    audio = input_stream.audio

    # Çıktı olarak kullanılacak ses dosyasını ve adını ayarla
    output_stream = ffmpeg.output(audio, audiofilename)

    # Eğer hedef dosya zaten varsa üzerine yaz
    output_stream = ffmpeg.overwrite_output(output_stream)

    # ffmpeg işlemini başlat ve ses dosyasını oluştur
    ffmpeg.run(output_stream)

    # Oluşturulan ses dosyasının adını döndür
    return audiofilename

def uploadFileAndGetTranscription(file_path, deepgram_api_key):
    """
    Ses dosyasını Deepgram API'sine yükler ve transkripsiyonunu alır.

    Args:
        file_path (str): Yüklenecek ses dosyasının yolu.
        deepgram_api_key (str): Deepgram API kullanımı için gerekli olan API anahtarı.

    Returns:
        dict: API'den dönen JSON yanıtını içeren bir sözlük. Genellikle transkripsiyon ve diğer ilgili bilgileri içerir.
    """

    # url = "https://api.deepgram.com/v1/listen"  # Deepgram API'nin ses dosyası dinleme endpoint'i
    url = "https://api.deepgram.com/v1/listen?model=whisper-large&language=en&punctuate=true&diarize=true&smart_format=true"


    headers = {
        "Authorization": f"Token {deepgram_api_key}"  # API anahtarını içeren yetkilendirme başlığı
    }

    # Ses dosyasını binary okuma modunda aç
    with open(file_path, 'rb') as audio_file:
        files = {
            'file': audio_file  # Ses dosyasını, istekle birlikte gönderilecek dosyaların bir parçası olarak ekleyin
        }

        # POST isteğini yap, dosyayı ve başlıkları gönder
        response = requests.post(url, headers=headers, files=files)
        # API'nin yanıtını JSON formatında dön
    return response.json()  # API yanıtını JSON olarak döndür


def convert_to_srt(data, output_filename):
    """
    Verilen veriyi SRT (SubRip Text) formatına dönüştürür ve bir dosyaya yazar.

    Args:
        data (list): Her biri 'start', 'end' ve 'word' anahtarlarına sahip sözlükler listesi.
                     'start' ve 'end', altyazının başlangıç ve bitiş zamanlarını saniye cinsinden belirtir.
                     'word', altyazı metnini içerir.
        output_filename (str): Oluşturulan SRT dosyasının kaydedileceği dosya adı.
    """

    def format_time(seconds):
        """
        Verilen süreyi saat:dakika:saniye,milisaniye formatına dönüştürür.

        Args:
            seconds (float): Dönüştürülecek süre, saniye cinsinden.

        Returns:
            str: "HH:MM:SS,mmm" formatında süre.
        """
        # Convert seconds to hours, minutes, seconds, milliseconds format
        hours, remainder = divmod(seconds, 3600)
        minutes, remainder = divmod(remainder, 60)
        seconds, milliseconds = divmod(remainder, 1)
        return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{int(milliseconds*1000):03d}"

    with open(output_filename, 'w') as f:
        for i, entry in enumerate(data, start=1):
            # Her bir girdi için başlangıç ve bitiş zamanlarını formatla
            start_time = format_time(entry['start'])
            end_time = format_time(entry['end'])
            # Altyazı metnini al
            subtitle_text = entry['word']
            # SRT formatında dosyaya yaz
            f.write(f"{i}\n")  # Sıra numarası
            f.write(f"{start_time} --> {end_time}\n")  # Başlangıç ve bitiş zamanları
            f.write(f"{subtitle_text}\n\n")  # Altyazı metni

# Search Functions

In [ ]:
def find_word_in_srt(search_word, srt_file_path="transcription.srt"):
    """
    Bir SRT dosyasında aranan kelimenin geçtiği tüm satırları bulur.
    - search_word: Aranacak kelime (string).
    - srt_file_path: SRT dosyasının yolu (string). Varsayılan değer 'transcription.srt'.
    - Büyük/küçük harf duyarlılığı olmadan arama yapılır.
    - Kelime tam eşleşmesi aranır (kelimenin tam olarak geçtiği yerler döndürülür).
    - Bulunan satırların temizlenmiş hali (başında/sonunda boşluk olmadan) döndürülür.
    """
    search_word = search_word.lower()  # Arama kelimesini küçük harfe çevir
    matches = []  # Eşleşmeleri saklamak için boş liste

    with open(srt_file_path, 'r', encoding='utf-8') as file:  # SRT dosyasını oku
        for line in file:  # Dosyayı satır satır oku
            line_lower = line.lower()  # Satırı küçük harfe çevir
            # Aranan kelimenin tam eşleşmesini ara (kelime sınırları içinde)
            if re.search(r'\b' + re.escape(search_word) + r'\b', line_lower):
                matches.append(line.strip())  # Eşleşen satırı listeye ekle

    return matches  # Eşleşen satırları döndür

def find_word_and_timestamp_in_srt(search_word, srt_file_path="transcription.srt"):
    """
    Aranan kelimenin geçtiği yerleri ve karşılık gelen zaman bilgilerini SRT dosyasından bulur.
    - search_word: Aranacak kelime (string).
    - srt_file_path: SRT dosyasının yolu (string). Varsayılan değer 'transcription.srt'.
    - Büyük/küçük harf duyarlılığı olmadan arama yapılır.
    - Kelimenin geçtiği yerler ve ilgili zaman damgaları bir liste olarak döndürülür.
    """
    search_word = search_word.lower()  # Arama kelimesini küçük harfe çevir
    results = []  # Sonuçları saklamak için boş liste

    with open(srt_file_path, 'r', encoding='utf-8') as file:
        content = file.read()  # Dosyanın tamamını oku

    subtitles = re.split(r'\n\n+', content)  # Alt yazı bloklarına ayır

    for subtitle in subtitles:  # Her alt yazı bloğu için
        lines = subtitle.split('\n')  # Alt yazıyı satırlara ayır
        if len(lines) < 3:  # Geçersiz alt yazı kontrolü
            continue

        time_info = lines[1]  # Zaman bilgisini al
        text = ' '.join(lines[2:]).lower()  # Metni birleştir ve küçük harfe çevir

        # Aranan kelimenin tam eşleşmesini metinde ara
        if re.search(r'\b' + re.escape(search_word) + r'\b', text):
            results.append((time_info, '\n'.join(lines[2:])))  # Zaman bilgisi ve metni sonuçlara ekle

    return results  # Sonuçları döndür

def time_to_seconds(time_str):
    """
    Zaman damgasını (string) saniye cinsinden sayıya dönüştürür.
    - time_str: 'saat:dakika:saniye' formatında zaman damgası (string).
    - Dönüş değeri: Zamanın saniye cinsinden karşılığı (int).
    """
    hours, minutes, seconds = [int(part) for part in time_str.split(':')[0:3]]  # Zamanı parçalara ayır ve int'e çevir
    seconds += 60 * minutes + 3600 * hours  # Toplam saniyeyi hesapla
    return seconds  # Saniye cinsinden değeri döndür

def find_word_and_seconds_in_srt(search_word, srt_file_path="transcription.srt"):
    """
    Aranan kelimenin geçtiği yerlerin zamanlarını saniye cinsinden bulur.
    - search_word: Aranacak kelime (string).
    - srt_file_path: SRT dosyasının yolu (string). Varsayılan değer 'transcription.srt'.
    - Büyük/küçük harf duyarlılığı olmadan arama yapılır.
    - Kelimenin geçtiği zamanlar (saniye cinsinden) bir liste olarak döndürülür.
    """
    search_word = search_word.lower()  # Arama kelimesini küçük harfe çevir
    seconds_list = []  # Zamanları saklamak için boş liste

    with open(srt_file_path, 'r', encoding='utf-8') as file:
        content = file.read()  # Dosyanın tamamını oku

    subtitles = re.split(r'\n\n+', content)  # Alt yazı bloklarına ayır

    for subtitle in subtitles:  # Her alt yazı bloğu için
        lines = subtitle.split('\n')  # Alt yazıyı satırlara ayır
        if len(lines) < 3:  # Geçersiz alt yazı kontrolü
            continue

        time_info = lines[1]  # Zaman bilgisini al
        text = ' '.join(lines[2:]).lower()  # Metni birleştir ve küçük harfe çevir

        # Aranan kelimenin tam eşleşmesini metinde ara
        if re.search(r'\b' + re.escape(search_word) + r'\b', text):
            start_time_str = time_info.split(' --> ')[0]  # Başlangıç zamanını al
            start_seconds = time_to_seconds(start_time_str.split(',')[0])  # Başlangıç zamanını saniyeye çevir
            seconds_list.append(start_seconds)  # Saniye cinsinden zamanı listeye ekle

    return seconds_list  # Zaman listesini döndür

# t

In [ ]:
deepgramapiKey = getpass('Enter your deepgram API key: ')

In [ ]:
soundFile = extract_sound_file(FILENAME)

In [ ]:
output = uploadFileAndGetTranscription(soundFile, deepgramapiKey)

In [ ]:
print(output)

In [ ]:
# Birinci Kod Parçası: Transkripti Yazdırma
# 'output' değişkeninden, ses tanıma sonucu elde edilen transkripti yazdırır.
# Bu kod, 'output' sözlüğündeki belirli bir yolu izleyerek, tanıma sonucu elde edilen metni alır ve yazdırır.
pprint(output['results']['channels'][0]['alternatives'][0]['transcript'])

# İkinci Kod Parçası: Kelime Detaylarını Yazdırma
# 'output' değişkeninden, ses tanıma sonucu elde edilen kelime detaylarını yazdırır.
# Bu kod, her bir kelimenin başlangıç zamanı, bitiş zamanı ve transkript içindeki kelimeyi içeren detaylı bilgileri alır ve yazdırır.
pprint(output['results']['channels'][0]['alternatives'][0]['words'])

In [ ]:
# 'output' değişkeninden kelime düzeyi zaman damgalarını içeren veriyi al
subtitle_data = output['results']['channels'][0]['alternatives'][0]['words']

# URL'den dosya adını çıkar
filename = os.path.basename(soundFile)
name, extension = os.path.splitext(filename)

# .srt uzantılı bir alt yazı dosyası adı oluştur
output_filename = name + ".srt"

# Oluşturulan veriyi kullanarak bir alt yazı (.srt) dosyası oluştur
convert_to_srt(subtitle_data, output_filename)

# Search

In [ ]:

# Belirtilen kelimenin SRT dosyasındaki geçtiği satırları bul
matches = find_word_in_srt(SEARCH, "siyasi.srt")

# Eğer eşleşme bulunduysa
if matches:
    print(f"'{SEARCH}' kelimesinin bulunduğu satırlar:")
    # Her eşleşmeyi ekrana yazdır
    for match in matches:
        print(match)
else:
    # Eğer eşleşme bulunamadıysa
    print(f"'{SEARCH}' kelimesi bulunamadı.")

In [ ]:
# Belirtilen kelimenin SRT dosyasındaki saniyelerini bul
seconds_list = find_word_and_seconds_in_srt(SEARCH, "siyasi.srt")

def sn_to_dk(num):
  dk = int(num/60)
  sn = num - (dk*60)
  return(f"{dk}.{sn}")

# Eğer saniyeler bulunduysa
if seconds_list:
    print(f"'{SEARCH}' kelimesinin bulunduğu dakikalar:")
    # Her bir saniyeyi ekrana yazdır
    for seconds in seconds_list:
      if seconds >= 60:
        print(f"[INFO] {sn_to_dk(seconds)} dk")
      else:
        print(f"[INFO] {seconds} sn")
else:
    # Eğer saniyeler bulunamadıysa
    print(f"'{SEARCH}' kelimesi bulunamadı.")

In [ ]:
# Belirtilen kelimenin SRT dosyasındaki zamanlar ve satırlarını bul
matches = find_word_and_timestamp_in_srt(SEARCH, "siyasi.srt")

# Eğer eşleşme bulunduysa
if matches:
    print(f"'{SEARCH}' kelimesinin bulunduğu zamanlar ve satırlar:")
    for time_info, match in matches:
        # Zaman bilgisini ve eşleşen metni ekrana yazdır
        print(f"Zaman: {time_info}")
        print(f"Metin: {match}\n")
else:
    # Eğer eşleşme bulunamadıysa
    print(f"'{SEARCH}' kelimesi bulunamadı.")